In [53]:
# !git clone https://github.com/QwenLM/Qwen3-TTS.git

In [55]:
# !pip install -e .

In [7]:
import json

def convert_manifest_for_qwen3(
    input_manifest,
    output_manifest,
    ref_audio_path
):
    """
    แปลง manifest เดิม -> format ที่ Qwen3-TTS official ต้องการ
    
    Parameters:
    - input_manifest: path ไฟล์ jsonl เดิม
    - output_manifest: path ไฟล์ jsonl ใหม่
    - ref_audio_path: path ของไฟล์ reference เสียง A (ใช้ไฟล์เดียวทั้ง dataset)
    """

    count = 0

    with open(input_manifest, "r", encoding="utf-8") as fin, \
         open(output_manifest, "w", encoding="utf-8") as fout:

        for line in fin:
            if not line.strip():
                continue

            item = json.loads(line)

            text = f"[{item['speaker']}] " + item["text"]

            new_item = {
                "audio": item["audio"],
                "text": text,
                "ref_audio": ref_audio_path[item['speaker']]
            }

            fout.write(json.dumps(new_item, ensure_ascii=False) + "\n")
            count += 1

    print(f"✅ Converted {count} samples")
    print(f"Saved to: {output_manifest}")

ref_dict = {"Lisa": "C:/Users/User/Comvi/Voice/Lisa/dataset_denoised/gEMrqw-pAy4/SPEAKER_01_sent_0001_DeepFilterNet3.wav",
                    "Bambam": "C:/Users/User/Comvi/Voice/Bambam/dataset_denoised/zbo7Mk3ryaI/SPEAKER_00_sent_0001_DeepFilterNet3.wav",
                    "IU": "C:/Users/User/Comvi/Voice/IU/dataset_denoised/wCbUWU4l_Ko/SPEAKER_01_sent_0000_DeepFilterNet3.wav",
                    "IVE": "C:/Users/User/Comvi/Voice/IVE/dataset_denoised/_037bSnAyRg/SPEAKER_01_sent_0007_DeepFilterNet3.wav"
                    }

convert_manifest_for_qwen3(
    input_manifest="C:/Users/User/Comvi/Voice/manifest_whisper_text.jsonl",
    output_manifest="C:/Users/User/Comvi/Voice/manifest_whisper_train_raw.jsonl",
    ref_audio_path=ref_dict
)

convert_manifest_for_qwen3(
    input_manifest="C:/Users/User/Comvi/Voice/manifest_pathumma_text.jsonl",
    output_manifest="C:/Users/User/Comvi/Voice/manifest_pathumma_train_raw.jsonl",
    ref_audio_path=ref_dict
)


✅ Converted 2558 samples
Saved to: C:/Users/User/Comvi/Voice/manifest_whisper_train_raw.jsonl
✅ Converted 2590 samples
Saved to: C:/Users/User/Comvi/Voice/manifest_pathumma_train_raw.jsonl


In [1]:
%cd Qwen3-TTS/finetuning


/workspace/Qwen3-TTS/finetuning


In [19]:
import os
import librosa
import soundfile as sf

def convert_to_24k(input_path, output_path=None):
    """
    Convert audio file to 24kHz mono WAV.

    Args:
        input_path (str): path to original audio file
        output_path (str, optional): output file path.
                                     If None, will overwrite with _24k.wav suffix

    Returns:
        str: path to converted file
    """

    # Load audio (auto-detect original sample rate)
    audio, sr = librosa.load(input_path, sr=None, mono=True)

    # Resample if needed
    if sr != 24000:
        audio = librosa.resample(audio, orig_sr=sr, target_sr=24000)
        sr = 24000

    # Set output path
    if output_path is None:
        base, ext = os.path.splitext(input_path)
        output_path = base + "_24k.wav"

    # Save as WAV 24kHz
    sf.write(output_path, audio, sr)

    return output_path

folder_list = ["C:/Users/User/Comvi/Voice/Lisa", "C:/Users/User/Comvi/Voice/Bambam", "C:/Users/User/Comvi/Voice/IU", "C:/Users/User/Comvi/Voice/IVE"]
file_path_list = []

for folder in folder_list:
    sub_folder = folder + "/dataset"
    for file in os.listdir(sub_folder):
        folder_path = folder + "/dataset/" + file
        for wav_file in os.listdir(folder_path):
            if wav_file.endswith(".wav"):
                file_path = folder_path + "/" + wav_file
                file_path_list.append(file_path)

for file_path in file_path_list:
    convert_to_24k(file_path, file_path)


In [28]:
import json
import soundfile as sf

manifest_path = "C:/Users/User/Comvi/Voice/manifest_whisper_train_raw.jsonl"

with open(manifest_path, "r", encoding="utf-8") as f:
    for line in f:
        data_dict = json.loads(line.strip())
        wav_path = data_dict["audio"]

        wav, sr = sf.read(wav_path)

        if sr != 24000:
            print("NOT 24K:", wav_path, "->", sr)

manifest_path = "C:/Users/User/Comvi/Voice/manifest_pathumma_train_raw.jsonl"

with open(manifest_path, "r", encoding="utf-8") as f:
    for line in f:
        data_dict = json.loads(line.strip())
        wav_path = data_dict["audio"]

        wav, sr = sf.read(wav_path)

        if sr != 24000:
            print("NOT 24K:", wav_path, "->", sr)

In [20]:
!python prepare_data.py --device cuda:0 --tokenizer_model_path Qwen/Qwen3-TTS-Tokenizer-12Hz --input_jsonl "C:/Users/User/Comvi/Voice/manifest_whisper_train_raw.jsonl" --output_jsonl "C:/Users/User/Comvi/Voice/manifest_whisper_train_with_code.jsonl"


********
********
 


'sox' is not recognized as an internal or external command,
operable program or batch file.
SoX could not be found!

    If you do not have SoX, proceed here:
     - - - http://sox.sourceforge.net/ - - -

    If you do (or think that you should) have SoX, double-check your
    path variables.
    


In [21]:
!python prepare_data.py --device cuda:0 --tokenizer_model_path Qwen/Qwen3-TTS-Tokenizer-12Hz --input_jsonl "C:/Users/User/Comvi/Voice/manifest_pathumma_train_raw.jsonl" --output_jsonl "C:/Users/User/Comvi/Voice/manifest_pathumma_train_with_code.jsonl"


********
********
 


'sox' is not recognized as an internal or external command,
operable program or batch file.
SoX could not be found!

    If you do not have SoX, proceed here:
     - - - http://sox.sourceforge.net/ - - -

    If you do (or think that you should) have SoX, double-check your
    path variables.
    


In [2]:
!pip install -U huggingface_hub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.3/553.3 kB 40.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 227.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11/11 [huggingface_hub] [huggingface_hub]


In [4]:
from huggingface_hub import snapshot_download

snapshot_download(
    repo_id="Qwen/Qwen3-TTS-12Hz-1.7B-Base",
    local_dir="./Qwen3-TTS-12Hz-1.7B-Base",
    local_dir_use_symlinks=False
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_validators.py:202: UserWarning: The `local_dir_use_symlinks` argument is deprecated and ignored in `snapshot_download`. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(


Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

'/workspace/Qwen3-TTS/finetuning/Qwen3-TTS-12Hz-1.7B-Base'

In [21]:
import json
import soundfile as sf

manifest_path = "../../manifest_whisper_train_with_code.jsonl"
output_path = "../../manifest_whisper_train_with_code_fixed.jsonl"

with open(manifest_path, "r", encoding="utf-8") as f_in, \
     open(output_path, "w", encoding="utf-8") as f_out:

    for line in f_in:
        data_dict = json.loads(line.strip())

        wav_path = data_dict["audio"]
        new_path = wav_path.replace("C:/Users/User/Comvi/Voice/", "../../")
        data_dict["audio"] = new_path

        ref_path = data_dict["ref_audio"]
        new_ref_path = ref_path.replace("C:/Users/User/Comvi/Voice/", "../../")
        data_dict["ref_audio"] = new_ref_path

        # เขียนกลับเป็น json 1 บรรทัด
        f_out.write(json.dumps(data_dict, ensure_ascii=False) + "\n")

print("Done. Saved to:", output_path)

manifest_path = "../../manifest_pathumma_train_with_code.jsonl"
output_path = "../../manifest_pathumma_train_with_code_fixed.jsonl"

with open(manifest_path, "r", encoding="utf-8") as f_in, \
     open(output_path, "w", encoding="utf-8") as f_out:

    for line in f_in:
        data_dict = json.loads(line.strip())

        wav_path = data_dict["audio"]
        new_path = wav_path.replace("C:/Users/User/Comvi/Voice/", "../../")
        data_dict["audio"] = new_path

        ref_path = data_dict["ref_audio"]
        new_ref_path = ref_path.replace("C:/Users/User/Comvi/Voice/", "../../")
        data_dict["ref_audio"] = new_ref_path

        # เขียนกลับเป็น json 1 บรรทัด
        f_out.write(json.dumps(data_dict, ensure_ascii=False) + "\n")

print("Done. Saved to:", output_path)

Done. Saved to: ../../manifest_whisper_train_with_code_fixed.jsonl
Done. Saved to: ../../manifest_pathumma_train_with_code_fixed.jsonl


In [24]:
!python sft_12hz.py --init_model_path "./Qwen3-TTS-12Hz-1.7B-Base" --output_model_path "../../output_whisper" --train_jsonl "../../manifest_whisper_train_with_code_fixed.jsonl" --batch_size 1 --lr 1e-6 --num_epochs 10 --speaker_name multi_speaker

/bin/sh: 1: sox: not found
SoX could not be found!

    If you do not have SoX, proceed here:
     - - - http://sox.sourceforge.net/ - - -

    If you do (or think that you should) have SoX, double-check your
    path variables.
    
/usr/local/lib/python3.11/dist-packages/accelerate/accelerator.py:529: UserWarning: `log_with=tensorboard` was passed but no supported trackers are currently installed.
  warnings.warn(f"`log_with={log_with}` was passed but no supported trackers are currently installed.")
`torch_dtype` is deprecated! Use `dtype` instead!
You are attempting to use Flash Attention 2 without specifying a torch dtype. This might lead to unexpected behaviour
Epoch 0 | Step 0 | Loss: 15.8386
Epoch 0 | Step 10 | Loss: 14.2030
Epoch 0 | Step 20 | Loss: 14.0107
Epoch 0 | Step 30 | Loss: 12.1982
Epoch 0 | Step 40 | Loss: 12.6076
Epoch 0 | Step 50 | Loss: 12.7813
Epoch 0 | Step 60 | Loss: 13.9061
Epoch 0 | Step 70 | Loss: 13.8017
Epoch 0 | Step 80 | Loss: 13.9025
Epoch 0 | Step 90 | 

In [ ]:
!python sft_12hz.py --init_model_path "./Qwen3-TTS-12Hz-1.7B-Base" --output_model_path "../../output_pathumma" --train_jsonl "../../manifest_pathumma_train_with_code_fixed.jsonl" --batch_size 1 --lr 1e-6 --num_epochs 10 --speaker_name multi_speaker

/bin/sh: 1: sox: not found
SoX could not be found!

    If you do not have SoX, proceed here:
     - - - http://sox.sourceforge.net/ - - -

    If you do (or think that you should) have SoX, double-check your
    path variables.
    
/usr/local/lib/python3.11/dist-packages/accelerate/accelerator.py:529: UserWarning: `log_with=tensorboard` was passed but no supported trackers are currently installed.
  warnings.warn(f"`log_with={log_with}` was passed but no supported trackers are currently installed.")
`torch_dtype` is deprecated! Use `dtype` instead!
You are attempting to use Flash Attention 2 without specifying a torch dtype. This might lead to unexpected behaviour
Epoch 0 | Step 0 | Loss: 18.4551
Epoch 0 | Step 10 | Loss: 14.4413
Epoch 0 | Step 20 | Loss: 12.2113
Epoch 0 | Step 30 | Loss: 14.2152
Epoch 0 | Step 40 | Loss: 14.2521
Epoch 0 | Step 50 | Loss: 12.7657
Epoch 0 | Step 60 | Loss: 14.6656
Epoch 0 | Step 70 | Loss: 12.3758
Epoch 0 | Step 80 | Loss: 12.6625
Epoch 0 | Step 90 | 

In [6]:
import torch
import soundfile as sf
from qwen_tts import Qwen3TTSModel

device = "cuda:0"

tts = Qwen3TTSModel.from_pretrained(
    "output_whisper/checkpoint-epoch-2",
    device_map=device,
    dtype=torch.float16,
)

ref_dict = {"Lisa": "C:/Users/User/Comvi/Voice/Lisa/dataset_denoised/gEMrqw-pAy4/SPEAKER_01_sent_0001_DeepFilterNet3.wav",
            "Bambam": "C:/Users/User/Comvi/Voice/Bambam/dataset_denoised/zbo7Mk3ryaI/SPEAKER_00_sent_0001_DeepFilterNet3.wav",
            "IU": "C:/Users/User/Comvi/Voice/IU/dataset_denoised/wCbUWU4l_Ko/SPEAKER_01_sent_0000_DeepFilterNet3.wav",
            "IVE": "C:/Users/User/Comvi/Voice/IVE/dataset_denoised/_037bSnAyRg/SPEAKER_01_sent_0007_DeepFilterNet3.wav"
            }

def generate_voice(speaker_name, text_input, output_file):
    text_input = f"[{speaker_name}] " + text_input

    wavs, sr = tts.generate_custom_voice(
        text=text_input,
        ref_audio=ref_dict[speaker_name],
        speaker=None
    )

    sf.write(output_file, wavs[0], sr)

# song1 = """ระหว่างที่มีดาวดวงนึงโคจรในอวกาศ..
# จากวันนั้นฉันก็มองโลกไม่เหมือนเดิม...
# สิ่งที่สวยงามในชีวิตพบเจอ..
# คงไม่เหลือคุณค่าอีกแล้ว.. ถูกมาแทนด้วยดวงตาคู่นั้นของ..เธอ...
# ปฏิทินจากนี้มีไว้เพื่อนับวัน.. นาฬิกามีไว้เฝ้าคอยนับนาที...
# ของชีวิตฉันนับต่อจากนี้ ที่จะมีไว้เพื่อเธอเท่านั้น...

# ขออยู่ในชีวิตที่เหลือของเธอได้ไหม..
# อยากลืมตาแล้วได้พบเธอจนวันสุดท้าย..
# อยากเป็นคนที่ได้นอนดูดาวข้างเธออีกหมื่นวัน
# และเอนไปจุมพิตเธอซักล้านครั้ง
# อยู่กับฉันไปนานๆ... นะเธอ.....

# ยังมีเพลงรักเป็นพันบทเพลงรอแชร์ให้เธอได้ฟัง..
# ยังมีอีกหลายทริปที่เรานั้นยังต้องไปเที่ยวด้วยกัน...
# ไม่รวมอีกเรื่องที่ยังสำคัญ ลูกเราที่สักวันจะเกิดมา
# ด้วยพันธสัญญาความรักของเรา...

# และมีดาวหางดวงนึงที่ยังโคจรในอวกาศ..
# ในช่วงชีวิตจะมีหนึ่งครั้งที่มองเห็นได้ด้วยตา..
# ดาวที่ฉันยังไม่เคยเห็นมาก่อน
# ขอให้ถึงวันนั้น
# ได้มีเธอรอดูมันด้วยกันกับฉัน...

# ขออยู่ในชีวิตที่เหลือของเธอได้ไหม..
# อยากลืมตาแล้วได้พบเธอจนวันสุดท้าย..
# อยากเป็นคนที่ได้นอนดูดาวข้างเธออีกหมื่นวัน
# และเอนไปจุมพิตเธอซักล้านครั้ง
# อยู่กับฉันไปนานๆ... นะเธอ.....


# ขออยู่ในชีวิตที่เหลือของเธอได้ไหม..
# อยากลืมตาแล้วได้พบเธอจนวันสุดท้าย..
# อยากเป็นคนที่ได้นอนดูดาวข้างเธออีกหมื่นวัน
# และเอนไปจุมพิตเธอซักล้านครั้ง
# อยู่กับฉันไปนานๆ... นะเธอ.....

# อยากเป็นคนที่ได้นอนดูดาวข้างเธออีกหมื่นวัน
# และเอนไปจุมพิตเธอซักล้านครั้ง.....
# อยู่กับฉันไปนานๆ... นะเธอ....."""

# song2 = """
# ได้มาพ้อ..น้องแก้มอ่องต่องในวันสงกรานต์..
# เต้นอยู่หน้าฮ้าน.. เมื่อคืนเทศกาลวันไหล...
# เจ้ายิ้มเข้ามาหยอก บอกว่าฮักอ้ายเบิดใจ..
# เข้ามาซบที่ตรงไหล่.. เจ้าว่าหัวใจเฮาตรงกัน

# อยู่กันสองคืน.. เจ้ามาขอกลับไปเฮ็ดงาน..
# ก่อนกระเจียวบาน.. บ่ต้องย่านสิกลับมาหา..
# อ้ายก็ถ่าจนเศร้า.. กอดรูปเจ้าทั้งน้ำตา..
# เจ้าคงลืมสัญญา..สามเดือนกว่ากว่าแล้วไปอยู่ไส....

# อยาก...เจอ ได้แค่เพียงคิดฮอด รอวันได้กอด...เธอ
# เฝ้าแต่ฝันละเมอกับคำว่าฮักที่เธอนั้นบอก
# อ้ายบ่ฮู้ว่าจริงหรือหยอก
# ย่านคนสวยน้องสิมาหลอกให้อ้ายช้ำ..

# ดอกกระเจียวบาน.. อีกไม่นานก็คงสิเฉา..
# อ้ายก็รอเจ้าอยู่คือเก่า ไปเป็นผู้สาวผู้ใดหนอ..นาง
# นั่ง...คึดฮอด..บ่ได้นอนจนฟ้าสาง..
# ย่านความฮักเฮาแตกม่าง
# เจ้าลืมทุกอย่างของสองเฮา...

# ดอกกระเจียวบาน.. ผ่านหน้าแล้งเจ้าไปอยู่ไส..
# เข้าหน้าฝนบ่โดนเท่าไหร่..
# น้ำตาของอ้ายกะไหลหย่าว...
# ใจสวอย..อย่าให้คอยถึงหน้าหนาว....
# จนดอกกระเจียวของอ้ายเหี่ยวเฉา
# เจ้ายังบ่..คืนมา.....

# อยาก...เจอ ได้แค่เพียงคิดฮอด รอวันได้กอด...เธอ
# เฝ้าแต่ฝันละเมอกับคำว่าฮักที่เธอนั้นบอก
# อ้ายบ่ฮู้ว่าจริงหรือหยอก
# ย่านคนสวยน้องสิมาหลอกให้อ้ายช้ำ....

# ดอกกระเจียวบาน.. อีกไม่นานก็คงสิเฉา..
# อ้ายก็รอเจ้าอยู่คือเก่า ไปเป็นผู้สาวผู้ใดหนอ..นาง
# นั่ง...คึดฮอด..บ่ได้นอนจนฟ้าสาง..
# ย่านความฮักเฮาแตกม่าง
# เจ้าลืมทุกอย่างของสองเฮา...

# ดอกกระเจียวบาน.. ผ่านหน้าแล้งเจ้าไปอยู่ไส..
# เข้าหน้าฝนบ่โดนเท่าไหร่..
# น้ำตาของอ้ายกะไหลหย่าว...
# ใจสวอย..อย่าให้คอยถึงหน้า....หนาว....
# จนดอกกระเจียวของอ้ายเหี่ยวเฉา
# เจ้ายังบ่..คืนมา........


# ดอกกระเจียวบาน.. ผ่านหน้าแล้งเจ้าไปอยู่ไส..
# เข้าหน้าฝนบ่โดนเท่าไหร่..
# น้ำตาของอ้ายกะไหลหย่าว...
# ใจสวอย..อย่าให้คอยถึงหน้า....หนาว....
# จนดอกกระเจียวของอ้ายเหี่ยวเฉา
# เจ้ายังบ่..คืนมา........

# จนกระเจียวของอ้ายเหี่ยวเฉา
# เจ้ายังบ่คืนมา...."""

# generate_voice("Lisa", song1, "C:/Users/User/Comvi/Voice/Lisa_song1_whisper_epoch-2.wav")
# generate_voice("Lisa", song2, "C:/Users/User/Comvi/Voice/Lisa_song2_whisper_epoch-2.wav")

# generate_voice("Bambam", song1, "C:/Users/User/Comvi/Voice/Bambam_song1_whisper_epoch-2.wav")
# generate_voice("Bambam", song2, "C:/Users/User/Comvi/Voice/Bambam_song2_whisper_epoch-2.wav")

# generate_voice("IU", song1, "C:/Users/User/Comvi/Voice/IU_song1_whisper_epoch-2.wav")
# generate_voice("IU", song2, "C:/Users/User/Comvi/Voice/IU_song2_whisper_epoch-2.wav")

# generate_voice("IVE", song1, "C:/Users/User/Comvi/Voice/IVE_song1_whisper_epoch-2.wav")
# generate_voice("IVE", song2, "C:/Users/User/Comvi/Voice/IVE_song2_whisper_epoch-2.wav")

In [9]:
song1 = """ขออยู่ในชีวิตที่เหลือของเธอได้ไหม..
อยากลืมตาแล้วได้พบเธอจนวันสุดท้าย..
อยากเป็นคนที่ได้นอนดูดาวข้างเธออีกหมื่นวัน
และเอนไปจุมพิตเธอซักล้านครั้ง
อยู่กับฉันไปนานๆ... นะเธอ....."""

song2 = """ดอกกระเจียวบาน.. อีกไม่นานก็คงสิเฉา..
อ้ายก็รอเจ้าอยู่คือเก่า ไปเป็นผู้สาวผู้ใดหนอ..นาง
นั่ง...คึดฮอด..บ่ได้นอนจนฟ้าสาง..
ย่านความฮักเฮาแตกม่าง
เจ้าลืมทุกอย่างของสองเฮา...

ดอกกระเจียวบาน.. ผ่านหน้าแล้งเจ้าไปอยู่ไส..
เข้าหน้าฝนบ่โดนเท่าไหร่..
น้ำตาของอ้ายกะไหลหย่าว...
ใจสวอย..อย่าให้คอยถึงหน้าหนาว....
จนดอกกระเจียวของอ้ายเหี่ยวเฉา
เจ้ายังบ่..คืนมา....."""



generate_voice("Lisa", song1, "C:/Users/User/Comvi/Voice/inference_audio/Lisa_song1_whisper_epoch-2.wav")
generate_voice("Lisa", song2, "C:/Users/User/Comvi/Voice/inference_audio/Lisa_song2_whisper_epoch-2.wav")

generate_voice("Bambam", song1, "C:/Users/User/Comvi/Voice/inference_audio/Bambam_song1_whisper_epoch-2.wav")
generate_voice("Bambam", song2, "C:/Users/User/Comvi/Voice/inference_audio/Bambam_song2_whisper_epoch-2.wav")

generate_voice("IU", song1, "C:/Users/User/Comvi/Voice/inference_audio/IU_song1_whisper_epoch-2.wav")
generate_voice("IU", song2, "C:/Users/User/Comvi/Voice/inference_audio/IU_song2_whisper_epoch-2.wav")

generate_voice("IVE", song1, "C:/Users/User/Comvi/Voice/inference_audio/IVE_song1_whisper_epoch-2.wav")
generate_voice("IVE", song2, "C:/Users/User/Comvi/Voice/inference_audio/IVE_song2_whisper_epoch-2.wav")

Setting `pad_token_id` to `eos_token_id`:2150 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2150 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2150 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2150 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2150 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2150 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2150 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2150 for open-end generation.


In [11]:
import torch
import soundfile as sf
from qwen_tts import Qwen3TTSModel

device = "cuda:0"

tts = Qwen3TTSModel.from_pretrained(
    "output_pathumma/checkpoint-epoch-2",
    device_map=device,
    dtype=torch.float16,
)

ref_dict = {"Lisa": "C:/Users/User/Comvi/Voice/Lisa/dataset_denoised/gEMrqw-pAy4/SPEAKER_01_sent_0001_DeepFilterNet3.wav",
            "Bambam": "C:/Users/User/Comvi/Voice/Bambam/dataset_denoised/zbo7Mk3ryaI/SPEAKER_00_sent_0001_DeepFilterNet3.wav",
            "IU": "C:/Users/User/Comvi/Voice/IU/dataset_denoised/wCbUWU4l_Ko/SPEAKER_01_sent_0000_DeepFilterNet3.wav",
            "IVE": "C:/Users/User/Comvi/Voice/IVE/dataset_denoised/_037bSnAyRg/SPEAKER_01_sent_0007_DeepFilterNet3.wav"
            }

def generate_voice(speaker_name, text_input, output_file):
    text_input = f"[{speaker_name}] " + text_input

    wavs, sr = tts.generate_custom_voice(
        text=text_input,
        ref_audio=ref_dict[speaker_name],
        speaker=None
    )

    sf.write(output_file, wavs[0], sr)

generate_voice("Lisa", song1, "C:/Users/User/Comvi/Voice/inference_audio/Lisa_song1_pathumma_epoch-2.wav")
generate_voice("Lisa", song2, "C:/Users/User/Comvi/Voice/inference_audio/Lisa_song2_pathumma_epoch-2.wav")

generate_voice("Bambam", song1, "C:/Users/User/Comvi/Voice/inference_audio/Bambam_song1_pathumma_epoch-2.wav")
generate_voice("Bambam", song2, "C:/Users/User/Comvi/Voice/inference_audio/Bambam_song2_pathumma_epoch-2.wav")

generate_voice("IU", song1, "C:/Users/User/Comvi/Voice/inference_audio/IU_song1_pathumma_epoch-2.wav")
generate_voice("IU", song2, "C:/Users/User/Comvi/Voice/inference_audio/IU_song2_pathumma_epoch-2.wav")

generate_voice("IVE", song1, "C:/Users/User/Comvi/Voice/inference_audio/IVE_song1_whisper_epoch-2.wav")
generate_voice("IVE", song2, "C:/Users/User/Comvi/Voice/inference_audio/IVE_song2_whisper_epoch-2.wav")

Setting `pad_token_id` to `eos_token_id`:2150 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2150 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2150 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2150 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2150 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2150 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2150 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2150 for open-end generation.
